In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

#　統計用ツール
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from patsy import dmatrices

#描画
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot

#株価
import pandas as pd
import pandas_datareader.data as web
import datetime

#深層学習
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

#k平均法
from sklearn.cluster import KMeans

In [4]:
s = (1990, 1, 1)
e = (2015, 9, 30)
t = 'Adj Close'
start = datetime.datetime(s[0], s[1], s[2])
end = datetime.datetime(e[0], e[1], e[2])
SaP = web.DataReader('GSPC', 'yahoo', start, end)[t]
N225 = web.DataReader('^N225', 'yahoo', start, end)[t]
DJI = web.DataReader('^DJI', 'yahoo', start, end)[t]
IXIC = web.DataReader('^IXIC', 'yahoo', start, end)[t]

In [5]:
SaP = np.array([SaP[1:].values - SaP[:-1].values]).T*100

In [6]:
def fuzzy(SaP):
    kms = KMeans(n_clusters=3).fit_predict(SaP)

    mean1 = SaP[kms == 0].mean()
    var1 = SaP[kms == 0].var()
    mean2 = SaP[kms == 1].mean()
    var2 = SaP[kms == 1].var()
    mean3 = SaP[kms == 2].mean()
    var3 = SaP[kms == 2].var()

    for t in range(100):
        f = SaP[t:t+20]

        v = np.array([])
        for i in range(20):
            v1 = np.round(np.exp(-1*(f[i] - mean1)**2/var1), 5)
            v2 = np.round(np.exp(-1*(f[i] - mean2)**2/var2), 5)
            v3 = np.round(np.exp(-1*(f[i] - mean3)**2/var3), 5)
            v = np.append(v, v1)
            v = np.append(v, v2)
            v = np.append(v, v3)

        if t != 0:
            fs = np.c_[fs, v]
        else:
            fs = np.array([v]).T
    return fs

In [87]:
class DRNN(object):
    def __init__(self, data):
        self.data = data
        self.model = DeepLearning()
        self.optimizer = optimizers.Adam()
        self.optimizer.setup(self.model)
        
    def fuzzy(self):
        self.fs = fuzzy(self.data).T
        
    def autoencorder(self, num):
        ft = np.array(self.fs, dtype='float32')
        
        n, m = ft.shape
        bs = 5
        
        for j in range(num):
            sffindx = np.random.permutation(n)
            for i in range(0, n, bs):
                self.ib = i+bs
                x = Variable(ft[sffindx[i:(i+bs) if (i+bs) < n else n]])
                self.model.zerograds()
                loss = self.model.ae(x)
                loss.backward()
                self.optimizer.update()
            if j % 1000 == 0:
                print('epoch:', j)
                print('train mean loss={}'.format(loss.data))
                print(' - - - - - - - - - ')
            
    def reinforcement(self, n):
        for i in range(n):
            self.optimizer = optimizers.Adam()
            self.optimizer.setup(self.model)
            for t in range(100):
                loss = self.model.rnn(self.fs, t)
                loss.backward()
                self.optimizer.update()
            print('epoch:', i)
            print('profit={}'.format(-loss.data[0][0]))
            print(' - - - - - - - - - ')
        
    def initialization(self, n=3000, m=100):
        self.fuzzy()
        self.autoencorder(n)
        self.reinforcement(m)
        
    def learning(self, maxc=10):
        c = 0
        while c < maxc:
            for t in range(100):
                loss = self.model(self.fs, t)
                loss.backward()
                self.optimizer.update()
                print('epoch:', c)
                print('time:', t)
                print('profit={}'.format(-loss.data[0][0]))
                print(' - - - - - - - - - ')
                c += 1
            
    def strategy():
        self.model.delta(self.fs)

In [88]:
class DeepLearning(chainer.Chain):
    def __init__(self):
        super(DeepLearning, self).__init__(
            w = L.Linear(20, 1),
            b = L.Linear(1, 1),
            u = L.Linear(1, 1),
            l1 = L.Linear(60, 60),
            l2 = L.Linear(60, 60),
            l3 = L.Linear(60, 60),
            l4 = L.Linear(60, 20),
            l5 = L.Linear(20, 60),
            m1 = L.Linear(1, 60),
            m2 = L.Linear(1, 60),
            m3 = L.Linear(1, 60),
            m4 = L.Linear(1, 20),
        )
    
    def ae(self, x):
        bv, fv = self.ae_fwd(x)
        loss = F.mean_squared_error(bv, x)
        return loss
    
    def ae_fwd(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = F.sigmoid(self.l2(h1))
        h3 = F.sigmoid(self.l3(h2))
        fv = F.sigmoid(self.l4(h3))
        bv = self.l5(fv)
        return bv, fv
    
    def rnn(self, fs, t, c=1):
        fs = np.array(fs, dtype='float32')
        bv, Fs = self.ae_fwd(Variable(fs))        
        Fs = Fs.data
        
        d_1 = Variable(np.zeros((1, 1), dtype=np.float32))
        old_grad = np.array([np.zeros(21)], dtype='float32')
        one = Variable(np.array([[1]], dtype=np.float32))
        zero = Variable(np.array([[0]], dtype=np.float32))

        loss = 0
        for i in range(len(Fs[:, 0])):
            x_k = Variable(np.array([Fs[i]], dtype=np.float32))
            d = F.tanh(self.w(x_k) + self.b(one) + self.u(d_1))
            z = Variable(np.array([[Fs[i][-1]]], dtype=np.float32))
            R = d_1.__rmatmul__(z) - c*(d - d_1).__abs__()
            loss += R
            d_1 = d
        return -loss
    
    def delta(self, Fs):
        one = Variable(np.array([[1]], dtype=np.float32))

        d_1 = Variable(np.zeros((1, 1), dtype=np.float32))
        d = np.zeros(100)
        d[0] = 1
        for i in range(1, len(Fs[:, 0])):
            Ft = Variable(np.array([Fs[i]], dtype=np.float32))
            ds = F.tanh(self.w(Ft) + self.b(one) + self.u(d_1))
            d[i] = ds.data[0]
        return d
        
    def __call__(self, fs, t, c=1):
        fs = np.array(fs, dtype='float32')
        bv, Fs = self.ae_fwd(Variable(fs))
        Fs_Var = Fs
        
        d_1 = Variable(np.zeros((1, 1), dtype=np.float32))
        old_grad = np.array([np.zeros(21)], dtype='float32')
        one = Variable(np.array([[1]], dtype=np.float32))
        zero = Variable(np.array([[0]], dtype=np.float32))
        loss = 0
        
        Fs = Fs.data
        for i in range(len(Fs[:, 0])):
            d = self.delta(Fs_Var.data).data
            if i == t:
                ft = Variable(np.array([fs[t]], dtype=np.float32))
                d = Variable(np.array([[d[t]]], dtype=np.float32))
                h1 = F.sigmoid(self.l1(ft) + self.m1(d)) 
                h2 = F.sigmoid(self.l2(h1) + self.m2(d))
                h3 = F.sigmoid(self.l3(h2) + self.m3(d))
                Ft = F.sigmoid(self.l4(h3) + self.m4(d)) 
                d = F.tanh(self.w(Ft) + self.b(one) + self.u(d_1))
                z = Variable(np.array([[Fs[i][-1]]], dtype=np.float32))
                R = d_1.__rmatmul__(z) - c*(d - d_1).__abs__()
                loss += R
                d_1 = d
            else:
                Ft = Variable(np.array([Fs[i]], dtype=np.float32))
                d = F.tanh(self.w(Ft) + self.b(one) + self.u(d_1))
                z = Variable(np.array([[Fs[i][-1]]], dtype=np.float32))
                R = d_1.__rmatmul__(z) - c*(d - d_1).__abs__()
                loss += R
                d_1 = d
        return -loss

In [91]:
drnn = DRNN(SaP)
drnn.initialization() 

epoch: 0
train mean loss=0.3211595118045807
 - - - - - - - - - 
epoch: 1000
train mean loss=0.008461804129183292
 - - - - - - - - - 
epoch: 2000
train mean loss=0.007551076356321573
 - - - - - - - - - 
epoch: 0
profit=51.91835403442383
 - - - - - - - - - 
epoch: 1
profit=95.23589324951172
 - - - - - - - - - 
epoch: 2
profit=97.85733795166016
 - - - - - - - - - 


KeyboardInterrupt: 

In [92]:
drnn.learning()

epoch: 0
time: 0
profit=97.97662353515625
 - - - - - - - - - 
epoch: 1
time: 1
profit=97.97737121582031
 - - - - - - - - - 
epoch: 2
time: 2
profit=97.97809600830078
 - - - - - - - - - 
epoch: 3
time: 3
profit=97.97891235351562
 - - - - - - - - - 
epoch: 4
time: 4
profit=97.979736328125
 - - - - - - - - - 


KeyboardInterrupt: 

In [93]:
bv, Ft = drnn.model.ae_fwd(np.array(drnn.fs, dtype=np.float32))
drnn.model.delta(Ft.data)

array([ 1.        ,  0.99993312,  0.99993312,  0.999933  ,  0.99993306,
        0.99993306,  0.99993312,  0.99993312,  0.99993306,  0.999933  ,
        0.99993312,  0.99993306,  0.99993312,  0.999933  ,  0.99993306,
        0.99993312,  0.99993318,  0.99993306,  0.99993312,  0.99993306,
        0.99993306,  0.99993312,  0.99993312,  0.99993312,  0.99993306,
        0.99993306,  0.99993312,  0.99993294,  0.99993312,  0.99993306,
        0.99993312,  0.99993306,  0.99993306,  0.99993312,  0.99993306,
        0.99993306,  0.99993306,  0.99993312,  0.99993312,  0.99993312,
        0.99993306,  0.99993306,  0.99993306,  0.99993306,  0.999933  ,
        0.99993294,  0.99993312,  0.99993312,  0.99993318,  0.99993312,
        0.999933  ,  0.99993306,  0.99993294,  0.999933  ,  0.999933  ,
        0.99993306,  0.99993318,  0.99993289,  0.99993306,  0.99993312,
        0.99993318,  0.999933  ,  0.99993306,  0.99993312,  0.999933  ,
        0.99993312,  0.99993312,  0.99993306,  0.99993294,  0.99